In [1]:
import pandas as pd
import data_util
from tec_an import TecAn
import numpy as np
from data_util import *
from bitstamp import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.neural_network import MLPClassifier

import xgboost as xgb


In [2]:
#live = LiveTrader()

In [3]:
#live.init()

In [4]:
#live.user_transactions(3)

In [5]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = 1234,
                 val_end = 1234,
                 train_start_list = []
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [6]:
win = 20
minus = 15

step =  2

tec = TecAn(windows = win, windows_limit = 100)
sourceDataGenerator = SourceDataGenerator(tec = tec)
online = get_online_data(minus, sourceDataGenerator, True, win)

In [7]:
cls = xgb.XGBClassifier(random_state=1,
                                   objective = "binary:logistic",
                                   eval_metric='mlogloss',
                                   learning_rate=0.01, 
                                   #use_label_encoder = True
                       )

In [8]:
x, y, close = train_by_step(cls, step, online)

In [9]:
print(Counter(y))

Counter({0: 6810, 1: 1182})


In [10]:
trainX_raw, trainY_raw = online.load_val_data("btcusd")
x, y, closed_prices = get_sequencial_data(trainX_raw, trainY_raw, step)


In [11]:
back = BackTest(value = 100, 
                verbose = True, 
                sell_on_profit = False,
                pending_sell_steps = step)


backtest_baseline(x, y, closed_prices, step, back)

BackTest (pending_sell_steps=2 sell_on_profit=False  value=100
1970-01-01 00:00:00 Buy (34256.42): ask: 34256.42
1970-01-01 00:00:00 Sell (34369.32) profit: 0.3296 total: 100.32957325955252
1970-01-01 00:00:00 Buy (33393.01): ask: 33393.01
1970-01-01 00:00:00 Sell (33521.96) profit: 0.3862 total: 100.71573191970325
1970-01-01 00:00:00 Buy (33462.1): ask: 33462.1
1970-01-01 00:00:00 Sell (33664.57) profit: 0.6051 total: 101.32080452423195
1970-01-01 00:00:00 Buy (33604.88): ask: 33604.88
1970-01-01 00:00:00 Sell (33711.95) profit: 0.3186 total: 101.63941896356337
1970-01-01 00:00:00 Buy (33246.09): ask: 33246.09
1970-01-01 00:00:00 Sell (34018.8) profit: 2.3242 total: 103.9636321266752
1970-01-01 00:00:00 Buy (33997.83): ask: 33997.83
1970-01-01 00:00:00 Sell (34081.75) profit: 0.2468 total: 104.21047141024123
1970-01-01 00:00:00 Buy (33851.04): ask: 33851.04
1970-01-01 00:00:00 Sell (34067.61) profit: 0.6398 total: 104.85024495929615
1970-01-01 00:00:00 Buy (33730.76): ask: 33730.76
19

1970-01-01 00:00:00 Buy (32936.12): ask: 32936.12
1970-01-01 00:00:00 Sell (33211.17) profit: 0.8351 total: 145.03644050176752
1970-01-01 00:00:00 Buy (32355.23): ask: 32355.23
1970-01-01 00:00:00 Sell (32567.59) profit: 0.6563 total: 145.69277952331055
1970-01-01 00:00:00 Buy (32504.12): ask: 32504.12
1970-01-01 00:00:00 Sell (32573.33) profit: 0.2129 total: 145.90570637689098
1970-01-01 00:00:00 Buy (32589.98): ask: 32589.98
1970-01-01 00:00:00 Sell (32705.07) profit: 0.3531 total: 146.25885173015598
1970-01-01 00:00:00 Buy (32646.25): ask: 32646.25
1970-01-01 00:00:00 Sell (32768.39) profit: 0.3741 total: 146.63298352170938
1970-01-01 00:00:00 Buy (32286.64): ask: 32286.64
1970-01-01 00:00:00 Sell (32476.75) profit: 0.5888 total: 147.2218029219319
1970-01-01 00:00:00 Buy (32501.04): ask: 32501.04
1970-01-01 00:00:00 Sell (32589.53) profit: 0.2723 total: 147.4940711324261
1970-01-01 00:00:00 Buy (32632.19): ask: 32632.19
1970-01-01 00:00:00 Sell (32703.09) profit: 0.2173 total: 147.7

In [12]:
back.current

156.41392201912907

In [13]:

closed_prices[:30]



array([34162.9 , 34269.56, 34220.15, 34174.91, 34216.83, 34256.42,
       34404.01, 34369.32, 34296.22, 34311.52, 34371.26, 34430.68,
       34472.48, 34489.11, 34256.8 , 33775.46, 33611.55, 33393.01,
       33529.58, 33521.96, 33524.7 , 33462.1 , 33683.71, 33664.57,
       33717.96, 33636.59, 33664.34, 33604.88, 33679.46, 33711.95])

In [14]:
back = BackTest(value = 100, 
                verbose = True, 
                sell_on_profit = True,
                pending_sell_steps = step)

back = backtest_model(cls, x, closed_prices, back)

BackTest (pending_sell_steps=2 sell_on_profit=True  value=100
1970-01-01 00:00:00 Buy (34162.9): ask: 34162.9
1970-01-01 00:00:00 Sell (34269.56) profit: 0.3122 total: 100.31221002900806
1970-01-01 00:00:00 Buy (34176.98): ask: 34176.98
1970-01-01 00:00:00 Sell (34191.56) profit: 0.0427 total: 100.35487032257406
1970-01-01 00:00:00 Buy (34433.55): ask: 34433.55
#### LOSSSS: -0.3171
1970-01-01 00:00:00 Sell (34324.35) profit: -0.3171 total: 100.03773775854854
#### LOSSSS: 0.0
1970-01-01 00:00:00 Sell (31321.95) profit: 0.0 total: 100.03773775854854


In [15]:
back.current

100.03773775854854